# <font size=6>Trabalho Final de Algoritmos Genéticos

___

<font size=3>Olá,</font> 

<font size=3>Este arquivo armazena as informações sobre o trabalho semestral desenvolvido na disciplina de **redes neurais e algoritmos genéticos por alunos da Ilum Escola de Ciência.**</font>

___

<font size=3>**Alunos:** Cauê Gomes Correia dos Santos, Izaque Junior Oliveira Silva e Karla Rovedo Pascoalini.</font> 

<font size=3>**Professor:** Daniel Roberto Cassar.</font> 
___

<font size=3>O intuito deste trabalho é desenvolver um `Algoritmo Genético` que otimize a função `minimiza_energia`, que busca a menor energia de uma molécula, de acordo com o método de Huckel, dado o número de átomos que a compõem, calculando todas as energias possíveis a partir de todas as combinações possíveis dado o número de átomos. A forma como essa função calcula a menor energia tende a ser custusa computacionalmente, como se pode imaginar. Devido a esse problema, uma otmização é necessária para que o custo computacional seja menor e possamos obter a menor energia da molécula almejada.</font>

## 1 - Importando as bibliotecas
___ 

In [1]:
from otimizando_menor_energia import create_upper_triangular_binary_matrix as create_traingular_umatrix
from otimizando_menor_energia import draw_graph as drawn
from otimizando_menor_energia import testar_matrizes
from otimizando_menor_energia import minimiza_energia
from otimizando_menor_energia import fitness as funcao_objetivo
from otimizando_menor_energia import initialize_population as inicializador#initialize_population
from otimizando_menor_energia import tournament_selection

In [13]:
import random
import numpy as np

<font size=3>A função `drawn` foi desenvolvida para plotar o grafo da molécula de menor energia que resultou da função `minimiza_energia` dado um conjunto de átomos.</font>

## 2 - Nobody knows how many possibles...
___

<font size=3>A função `minimiza_energia` recebe a quantidade de átomos que estarão presentes na molécula. A partir desses dados ela calcula a quantidade de possíveis arranjos entre esse átomos e gera todas as possíveis matrizes, que são `binárias`. Com as matrizes geradas na função acima, a função `testa_matrizes` recebe as matrizes e calcula a energia total (`t_total`) de cada matriz e rotorna a `melhor matriz`, junto a seus `autovalores (em ordem crescente)` e a `menor energia encontrada`.</font>

In [4]:
matrizes = minimiza_energia(6)
A = testar_matrizes(matrizes)

G=nx.from_numpy_array(A)
#nx.draw(G, with_labels=True)
#draw_graph(G)

------------------------------------------------------------ Resultado ------------------------------------------------------------
A matriz com o menor t_total associado é:
[[ 0.  0. -1. -1.  0. -1.]
 [ 0.  0.  0. -1. -1. -1.]
 [-1.  0.  0.  0. -1. -1.]
 [-1. -1.  0.  0.  0. -1.]
 [ 0. -1. -1.  0.  0. -1.]
 [-1. -1. -1. -1. -1.  0.]]
Com os autovalores [-3.4494897427831814, -0.6180339887498958, -0.6180339887498947, 1.4494897427831785, 1.6180339887498956, 1.6180339887498956]
E com o valor associado de -9.371115440565944


## 3 - Talvez o acaso me ajude...
___

<font size=3>Na tentativa de resolver o problema de encontrar a menor energia com um custo computacional menor, desenvolvemos um código de algoritmo genético para otimizar o nosso objetivo, cuja `função objetivo` é encontrar a energia total. Ao final, o algoritmo nos entregará o indivíduo com a menor `energia total` encontrada.</font>

<font size=3>Nosso código contou com alterações nos operadores genéticos `cruzamento` e `mutação`, os quais adaptamos para resolver o nosso problema. Os operadores utilizados foram:</font>

<font size=3>- Seleção: seleção por torneio com 3 indivíduos.</font>


<font size=3>- Cruzamento: um novo cruzamento (verificar isso) que apenas altera a triangular superior ou inferior de uma matriz.</font>


<font size=3>- Mutação: mutação simples, porém com a restrição de somente ocorrer fora e acima da diagonal principal.</font>


<font size=3>- Hall da Fama: indivíduo com a menor energia total encontrada.</font>


In [21]:
#Mutacao (aparentemente atualizado)
# Mutate the matrix by flipping a randomly chosen element in the upper triangular part (excluding the main diagonal)
def mutate_upper_triangular_binary(individuos, chance_mutacao):
    '''
    Muta uma matrix triangular superior, trocando um elemento 
    aleatório (fora da diagonal) de 0 para 1, ou vice-versa.

    Arg: 
        individuos: lista contendo as matrizes triangulares superiores
        chance_mutacao: float que define o limiar da chance de ocorrer mutação no indivíduo
        
    return: 
        retorna uma matriz triangular superior com uma mutação
    '''
    size = 0
    for individuo in individuos:
        size = individuo.shape[0]
        if random.random() < chance_mutacao:

            # Escolhe um elemento aleatório (tirando a diagonal)
            i = np.random.randint(0, size - 1)
            j = np.random.randint(i + 1, size)  # Assegura que estará acima da diagonal principal

            # Flipa o valor do elemento escolhido
            individuo[i, j] = 1 - individuo[i, j]

In [14]:
# Crossover function
def crossover(parent1, parent2, chance_cruzamento):
    '''
    Realiza o cruzamento entre dois pais para gerar um filho.

    Args:
        parent1: Tupla contendo a matriz triangular superior e a matriz simétrica correspondente do primeiro pai.
        parent2: Tupla contendo a matriz triangular superior e a matriz simétrica correspondente do segundo pai.
    
    return: 
        retorna uma tupla contendo a matriz triangular superior e a matriz simétrica correspondente do filho gerado.
    '''
    
    upper_tri1, _ = parent1
    upper_tri2, _ = parent2
    if random.random() < chance_cruzamento:
        child1 = []
        child2 = []
        size = upper_tri1.shape[0]
        
        crossover_point = np.random.randint(1, size)
        
        child1 = np.zeros((size, size)) #criando matriz de zeros para o filho 1
        child2 = np.zeros((size, size)) #criando matriz de zeros para o filho 2
        

        # Copy upper part from parent1 and lower part from parent2
        for i in range(size):
            for j in range(size):
                if j >= crossover_point:
                    child1[i, j] = upper_tri1[i, j]
                    child2[i, j] = upper_tri2[i, j]
                else:
                    child1[i, j] = upper_tri2[i, j]
                    child2[i, j] = upper_tri2[i, j]

        child1_combined = child1 + child1.T
        child2_combined = child2 + child2.T
        
        filho1 = (child1, child1_combined)
        filho2 = (child2, child2_combined)
        
        return filho1, filho2
    
    else:
        return parent1, parent2
    

In [38]:
# Genetic Algorithm
def genetic_algorithm(pop_size, matrix_size, generations, chance_mutacao, chance_cruzamento):
    # Initialize population
    population = inicializador(pop_size, matrix_size)
    
    for generation in range(generations):
        # Evaluate fitness of the population
        fitnesses = [funcao_objetivo(combined_matrix) for _, combined_matrix in population]
        
        new_population = []
        
        # Generate new population
        for _ in range(pop_size // 2):
            # Select parents
            parent1 = tournament_selection(population, fitnesses)
            parent2 = tournament_selection(population, fitnesses)
            
            # Crossover
            child1, child2 = crossover(parent1, parent2, chance_cruzamento)
            
            
            # Mutate
            lista_filhos = [child1[0], child2[0]]
            
            mutate_upper_triangular_binary(lista_filhos, chance_mutacao)#, child1[0] + child1[0].T)
            #child2 = (mutate_upper_triangular_binary(child2[0]), child2[0] + child2[0].T)
            
            new_population.extend([child1, child2])
            #print(new_population)
        
        # Replace old population with new population
        population = new_population
    
    # Final population fitness evaluation
    fitnesses = [funcao_objetivo(combined_matrix) for _, combined_matrix in population]
    best_individual = population[np.argmin(fitnesses)]
    
    return best_individual, fitnesses

In [48]:
pop_size = 100
matrix_size = 7
generations = 80
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.2
# Run genetic algorithm
best_solution, fit = genetic_algorithm(pop_size, matrix_size, generations, CHANCE_MUTACAO, CHANCE_CRUZAMENTO)

print("Best solution found:")
print(best_solution[1])  # Print the combined matrix
print("Fitness of the best solution:", funcao_objetivo(best_solution[1]))

Best solution found:
[[0. 1. 1. 1. 0. 0. 0.]
 [1. 0. 1. 1. 1. 1. 0.]
 [1. 1. 0. 0. 1. 0. 1.]
 [1. 1. 0. 0. 0. 1. 1.]
 [0. 1. 1. 0. 0. 1. 1.]
 [0. 1. 0. 1. 1. 0. 1.]
 [0. 0. 1. 1. 1. 1. 0.]]
Fitness of the best solution: -11.069194527751078


## Referências Bibliográficas

[1] Cassar, Daniel Roberto. GA 4.2 - Notebook Descobrindo a senha.

[2] Santos, Cauê Gomes Correia dos. 